# Chapter 13: Sampling Plans

In [1]:
import numpy as np

## Algorithm 13.1

In [9]:
def samples_full_factorial(a, b, m):
    ranges = (np.linspace(a[i], b[i], m[i] ) for i in range(len(a)))
    return np.meshgrid(*ranges)

## Algorithm 13.2

In [19]:
def uniform_projection_plan(m, n):
    perms = [ np.random.permutation(m) for i in range(n) ]
    return [ [perms[i][j] for i in range(n)] for j in range(m) ]

## Algorithm 13.3

In [2]:
def pairwise_distances(X, p=2):
    m = len(X)
    return [np.linalg.norm(X[i]-X[j],p) for i in range(m-1) for j in range(i+1,m)]

### Example

In [5]:
X = np.array([[1.0, 3.0], [2.0, 4.1], [9.0,1.0]])
pairwise_distances(X)

[1.4866068747318502, 8.246211251235321, 7.6557168181692825]